In [1]:
import nltk
nltk.data.path.append("nltk_data")
from nltk.corpus import gutenberg
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy

In [2]:
#compute TD.IDF for 3 files

texts = gutenberg.fileids()[:3]
files = []
for text in texts:
    files.append(gutenberg.raw(text).replace('\n', ' ').replace('_', ''))
#     print(files[text])
    
vectorizer = TfidfVectorizer(stop_words='english',min_df=1)
X = vectorizer.fit_transform(files)
words = vectorizer.get_feature_names()
# print (words) # print all the words
# numpy.set_printoptions(threshold=numpy.nan)1 #use to view all X
print (numpy.around(X.toarray(),8))

[[ 0.00104886  0.00061947  0.         ...,  0.          0.          0.00052443]
 [ 0.          0.00065283  0.00084064 ...,  0.00168129  0.00084064  0.        ]
 [ 0.          0.00042669  0.00054944 ...,  0.00109887  0.00054944  0.        ]]


In [3]:
#compute the most representative 10 words in 3 files
i = 0
for file in files:
    # get array of word importance
    importance = numpy.array(X.toarray()[i])
    
    # sort word importance
    sorted_vals = numpy.argsort(importance)
    reverse_sorted_vals = sorted_vals[::-1]
    top_reverse_sorted_vals = reverse_sorted_vals[:10]
    
    #print results
    print(texts[i])
    for index in top_reverse_sorted_vals:
        print (words[index], ' - ', importance[index])
    print ('\n')
    
    i += 1

austen-emma.txt
mr  -  0.357436692024
emma  -  0.344999193767
harriet  -  0.265361890146
weston  -  0.230749469692
mrs  -  0.21712575486
knightley  -  0.204003508432
elton  -  0.202954647206
miss  -  0.186461775215
woodhouse  -  0.164671212462
said  -  0.149912789376


austen-persuasion.txt
anne  -  0.324458969468
elliot  -  0.319444893613
captain  -  0.254715089464
wentworth  -  0.240965352276
mrs  -  0.189974970051
charles  -  0.183487378338
mr  -  0.167125746849
russell  -  0.163591156591
walter  -  0.155853737022
musgrove  -  0.143694934843


austen-sense.txt
elinor  -  0.494873206804
marianne  -  0.408902532921
mrs  -  0.226143717665
dashwood  -  0.182055544693
said  -  0.169394445119
jennings  -  0.166161806664
willoughby  -  0.156047609737
edward  -  0.144501733023
lucy  -  0.134374330607
sister  -  0.120325525248




In [4]:
#prepare for where-was-that function, load corpus files
texts = gutenberg.fileids()
files = []
for text in texts:
    files.append(gutenberg.raw(text).replace('\n', ' ').replace('_', ''))
vectorizer = TfidfVectorizer(stop_words='english',min_df=1)

#where-was-that function
def find( str ):
    files.append(str)
    X = vectorizer.fit_transform(files)
    # no need to normalize, since Vectorizer will return normalized tf-idf

    #remove noise
    X = numpy.around(X.toarray(),2)
    # compute the dot croduct (cosine similarity)
    pairwise_similarity = numpy.dot(X, numpy.transpose(X))
    #choose only the required result (input file)
    similarity = pairwise_similarity[len(files)-1][:len(files)-1]
    
    #sort the results
    sorted_vals = numpy.argsort(similarity)
    reverse_sorted_vals = sorted_vals[::-1]
    top_reverse_sorted_vals = reverse_sorted_vals[:10]
    
    #check for obvious result
    if similarity[top_reverse_sorted_vals[0]]/similarity[top_reverse_sorted_vals[1]] > 2:
        print ("The first result is the clear choice:")
    else:
        print ("There is no clear choice, but first is preffered:")

    #print results
    i = 1
    for index in top_reverse_sorted_vals:
        print(i, ". ", texts[index], " - ", similarity[index])
        i +=1
    
    #remove input file in order to reuse files list
    files.remove(str)
    
    print("\n")

In [5]:
find("story with the sailor and the whale")
find("story with the girl falling into a rabbit hole")
find("story with the girl falling into a rabbit hole Alice")
find("man walk story now then ago time")

The first result is the clear choice:
1 .  melville-moby_dick.txt  -  0.3534
2 .  bryant-stories.txt  -  0.024
3 .  chesterton-brown.txt  -  0.0174
4 .  whitman-leaves.txt  -  0.0066
5 .  edgeworth-parents.txt  -  0.0036
6 .  carroll-alice.txt  -  0.0036
7 .  burgess-busterbrown.txt  -  0.0
8 .  austen-persuasion.txt  -  0.0
9 .  austen-sense.txt  -  0.0
10 .  bible-kjv.txt  -  0.0


There is no clear choice, but first is preffered:
1 .  carroll-alice.txt  -  0.0476
2 .  bryant-stories.txt  -  0.0397
3 .  chesterton-brown.txt  -  0.0274
4 .  burgess-busterbrown.txt  -  0.0126
5 .  edgeworth-parents.txt  -  0.0121
6 .  whitman-leaves.txt  -  0.0117
7 .  blake-poems.txt  -  0.0086
8 .  chesterton-ball.txt  -  0.0083
9 .  melville-moby_dick.txt  -  0.0075
10 .  chesterton-thursday.txt  -  0.0043


The first result is the clear choice:
1 .  carroll-alice.txt  -  0.5139
2 .  bryant-stories.txt  -  0.0311
3 .  chesterton-brown.txt  -  0.0212
4 .  burgess-busterbrown.txt  -  0.01
5 .  edgewor